In [1]:
import pandas as pd
import numpy as np
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
print("Setup Complete")

Setup Complete


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# Read the file into a variable fifa_data
filepath = "/home/ryan/stark/MachineHack/Food_QUalityA_ParticipantsData/Data_Train.xlsx"
data = pd.read_excel(filepath)


In [3]:
testdata = pd.read_excel('/home/ryan/stark/MachineHack/Food_QUalityA_ParticipantsData/Data_Test.xlsx')


In [4]:
data.head()

,ID,Date,LicenseNo,FacilityID,FacilityName,Type,Street,City,State,LocationID,Reason,SectionViolations,RiskLevel,Geo_Loc,Inspection_Results
0,31103489027986,26-04-2010,4744,8123,7715,RESTAURANT,15522,id-11235901,id_1890134,81876.0,CANVASS,33.0,High,locid16406,4
1,10088999935915,21-06-2009,2973,12268,11664,GROCERY STORE,3057,id-11235901,id_1890134,81862.0,COMPLAINT,33.0,High,locid878,4
2,40148966010272,01-05-2013,18223,1112,969,RESTAURANT,14988,id-11235901,id_1890134,81883.0,CANVASS,NaN,High,locid3368,6
3,37157708563676,28-09-2015,20825,20007,19115,RESTAURANT,3661,id-11235901,id_1890134,81859.0,CANVASS RE-INSPECTION,31.0,Medium,locid11839,4
4,47478049564374,09-12-2015,2136,16867,10409,RESTAURANT,7876,id-11235901,id_1890134,81886.0,COMPLAINT,30.0,High,locid12264,4


In [5]:
# from datetime import datetime
# data['Date'] = pd.to_datetime(data['Date'] ,errors='coerce')

In [6]:
data['Geo_Loc'] = data['Geo_Loc'].str.replace(r'\D', '')

In [7]:
data['Geo_Loc'] = pd.to_numeric(data['Geo_Loc'], errors='coerce') 


In [8]:
data['Geo_Loc'].dtype

dtype('int64')

In [9]:
data['Geo_Loc'].head()

0    16406
1      878
2     3368
3    11839
4    12264
Name: Geo_Loc, dtype: int64

In [10]:
total = data.isnull().sum().sort_values(ascending=False)
percent_1 = data.isnull().sum()/data.isnull().count()*100
percent_2 = (np.round(percent_1, 1)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%']) #ptr
missing_data


,Total,%
SectionViolations,39068,26.5
Type,3485,2.4
LocationID,35,0.0
Inspection_Results,0,0.0
Geo_Loc,0,0.0
RiskLevel,0,0.0
Reason,0,0.0
State,0,0.0
City,0,0.0
Street,0,0.0


In [11]:
from sklearn.model_selection import train_test_split
y = data.Inspection_Results

# To keep things simple, we'll use only numerical predictors
melb_predictors = data.drop(['Inspection_Results'], axis=1)
X = melb_predictors.copy() 
# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.80, test_size=0.20,random_state=0)

In [12]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import log_loss
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report
    
# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestClassifier(random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    target_names = ['class 0', 'class 1', 'class 2','class 3', 'class 4', 'class 5', 'class 6']
    print(classification_report(y_valid, preds, target_names=target_names,labels= [0,1,2,3,4,5,6]))
    return model.score(X_valid,y_valid) 

In [13]:
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer(strategy='most_frequent')
imX_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imX_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation removed column names; put them back
imX_train.columns = X_train.columns
imX_valid.columns = X_valid.columns

In [14]:
imX_train.dtypes

ID                   object
Date                 object
LicenseNo            object
FacilityID           object
FacilityName         object
Type                 object
Street               object
City                 object
State                object
LocationID           object
Reason               object
SectionViolations    object
RiskLevel            object
Geo_Loc              object
dtype: object

In [15]:
categorical_cols = [cname for cname in X.columns if X[cname].nunique() < 500000000000000000 and 
                        X[cname].dtype == "object"]

In [16]:
categorical_cols

['Date', 'Type', 'City', 'State', 'Reason', 'RiskLevel']

In [17]:
numerical_cols = [cname for cname in X.columns if X[cname].dtype in ['int64', 'float64']]


In [18]:
numerical_cols

['ID',
 'LicenseNo',
 'FacilityID',
 'FacilityName',
 'Street',
 'LocationID',
 'SectionViolations',
 'Geo_Loc']

In [19]:
imX_train[numerical_cols] = imX_train[numerical_cols].apply(pd.to_numeric, errors='coerce')
imX_valid[numerical_cols] = imX_valid[numerical_cols].apply(pd.to_numeric, errors='coerce')


In [20]:
imX_train.dtypes

ID                     int64
Date                  object
LicenseNo              int64
FacilityID             int64
FacilityName           int64
Type                  object
Street                 int64
City                  object
State                 object
LocationID           float64
Reason                object
SectionViolations    float64
RiskLevel             object
Geo_Loc                int64
dtype: object

In [21]:
from datetime import datetime
imX_train['Date'] = pd.to_datetime(imX_train['Date'] ,errors='coerce')
imX_valid['Date'] = pd.to_datetime(imX_valid['Date'] ,errors='coerce')

In [22]:
imX_train.dtypes

ID                            int64
Date                 datetime64[ns]
LicenseNo                     int64
FacilityID                    int64
FacilityName                  int64
Type                         object
Street                        int64
City                         object
State                        object
LocationID                  float64
Reason                       object
SectionViolations           float64
RiskLevel                    object
Geo_Loc                       int64
dtype: object

In [23]:
s = (imX_train.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

Categorical variables:
['Type', 'City', 'State', 'Reason', 'RiskLevel']


In [25]:
from sklearn.preprocessing import LabelEncoder

# Make copy to avoid changing original data 
label_X_train = imX_train.copy()
label_X_valid = imX_valid.copy()

# Apply label encoder to each column with categorical data
label_encoder = LabelEncoder()
for col in object_cols:
    label_X_train[col] = label_encoder.fit_transform(imX_train[col])
    label_X_valid[col] = label_encoder.transform(imX_valid[col])



ValueError: y contains previously unseen labels: 'WHOLESALE & RETAIL'

In [ ]:
# (data
#  .groupby(['Date', 'Inspection_Results'])
#  .size()
#  .unstack()
#  .plot.bar()
# )

In [ ]:
label_X_train = label_X_train.assign(hour=label_X_train.Date.dt.hour,
               day=label_X_train.Date.dt.day,
               month=label_X_train.Date.dt.month,
               year=label_X_train.Date.dt.year)

In [ ]:
label_X_valid = label_X_valid.assign(hour=label_X_valid.Date.dt.hour,
               day=label_X_valid.Date.dt.day,
               month=label_X_valid.Date.dt.month,
               year=label_X_valid.Date.dt.year)

In [ ]:
label_X_train = label_X_train.drop(['Date','ID'] , axis=1)

In [ ]:
label_X_valid = label_X_valid.drop(['Date','ID' ], axis=1)

In [ ]:
label_X_train.isnull().sum()

In [ ]:
label_X_train.fillna(label_X_train.mean(), inplace=True)
label_X_valid.fillna(label_X_valid.mean(), inplace=True)


In [ ]:
#error on validation data
score_dataset(label_X_train, label_X_valid, y_train, y_valid)

In [ ]:
#error on traning data
score_dataset(label_X_train,label_X_train , y_train, y_train)

# Feature Engineering

In [ ]:
lxt = label_X_train.copy()
lxv = label_X_valid.copy()

In [ ]:
lxt

In [ ]:
interactions = (lxt['Reason'].map(str)) + "_" + (lxt['RiskLevel'].map(str))
len(interactions.unique())

In [ ]:
label_enc = LabelEncoder()
traindata_interaction = lxt.assign(ReasonRiskLevel=label_enc.fit_transform(interactions))
traindata_interaction.head()

In [ ]:
testdata_interaction = lxv.assign(ReasonRiskLevel=interactions)


In [ ]:
testdata_interaction['ReasonRiskLevel'] = label_enc.transform(testdata_interaction['ReasonRiskLevel'])

In [ ]:
# traindata_interaction['ReasonRiskLevel']

In [ ]:
testdata_interaction.head()

In [ ]:
##feature2

In [ ]:
interactions2 = (traindata_interaction['Reason'].map(str)) + "_" + (traindata_interaction['SectionViolations'].map(str))
len(interactions2.unique())

In [ ]:
label_enc2 = LabelEncoder()
traindata_interaction = traindata_interaction.assign(ReasonSectionViolations=label_enc2.fit_transform(interactions2))
traindata_interaction.head()

In [44]:
testdata_interaction = testdata_interaction.assign(ReasonSectionViolations=interactions2)

In [45]:
testdata_interaction['ReasonSectionViolations'] = label_enc2.transform(testdata_interaction['ReasonSectionViolations'])

In [46]:
####feature 3

In [47]:
interactions3 = (traindata_interaction['SectionViolations'].map(str)) + "_" + (traindata_interaction['RiskLevel'].map(str))




In [48]:
label_enc3 = LabelEncoder()
traindata_interaction = traindata_interaction.assign(SectionViolationsRiskLevel=label_enc3.fit_transform(interactions3))


In [49]:
testdata_interaction = testdata_interaction.assign(SectionViolationsRiskLevel=interactions3)
testdata_interaction['SectionViolationsRiskLevel'] = label_enc3.transform(testdata_interaction['SectionViolationsRiskLevel'])

In [50]:
testdata_interaction.head()

,LicenseNo,FacilityID,FacilityName,Type,Street,City,State,LocationID,Reason,SectionViolations,RiskLevel,Geo_Loc,hour,day,month,year,ReasonRiskLevel,ReasonSectionViolations,SectionViolationsRiskLevel
0,28225,4702,4480,326,13072,0,0,81875.0,0,2.0,0,15302,0,7,9,2018,41,496,72
1,743,25793,24635,53,2087,0,0,81882.0,0,32.0,2,3211,0,17,11,2014,0,26,76
2,24297,13257,12588,326,2382,0,0,81855.0,1,32.0,0,9774,0,19,12,2015,0,13,36
3,18030,7609,7234,351,12441,0,0,81883.0,0,32.0,0,3875,0,21,10,2010,28,333,63
4,17544,23678,22588,326,479,0,0,81862.0,0,33.0,0,511,0,31,1,2012,25,275,72


In [51]:
#error on validation data
score_dataset(traindata_interaction, testdata_interaction, y_train, y_valid)

              precision    recall  f1-score   support

     class 0       0.00      0.00      0.00         0
     class 1       0.30      0.35      0.32        23
     class 2       0.00      0.00      0.00         0
     class 3       0.00      0.00      0.00         4
     class 4       0.79      0.84      0.81        86
     class 5       0.54      0.57      0.55        23
     class 6       0.50      0.25      0.33        12

   micro avg       0.65      0.65      0.65       148
   macro avg       0.30      0.29      0.29       148
weighted avg       0.63      0.65      0.64       148



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.6486486486486487

In [81]:
#error on traning data
score_dataset(traindata_interaction, traindata_interaction, y_train, y_train)

              precision    recall  f1-score   support

     class 0       1.00      0.98      0.99        45
     class 1       1.00      1.00      1.00     28433
     class 2       1.00      1.00      1.00      1420
     class 3       1.00      1.00      1.00      4743
     class 4       1.00      1.00      1.00     79367
     class 5       1.00      1.00      1.00     20608
     class 6       1.00      1.00      1.00     12679

    accuracy                           1.00    147295
   macro avg       1.00      1.00      1.00    147295
weighted avg       1.00      1.00      1.00    147295



0.999850639872365

# Feature selection 
## l1 penalty

In [52]:
from sklearn.linear_model import LogisticRegression

from sklearn.feature_selection import SelectFromModel


# Set the regularization parameter C=1
logistic = LogisticRegression(C=1, penalty="l1", random_state=7 ,solver='liblinear').fit(traindata_interaction, y_train)
model = SelectFromModel(logistic, prefit=True)


X_new = model.transform(traindata_interaction)
X_new

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


array([[2.6121e+04, 7.6280e+03, 7.2520e+03, ..., 4.1000e+01, 4.9600e+02,
        7.2000e+01],
       [3.6379e+04, 9.2940e+03, 8.8180e+03, ..., 0.0000e+00, 2.6000e+01,
        7.6000e+01],
       [2.0219e+04, 4.6420e+03, 4.4290e+03, ..., 0.0000e+00, 1.3000e+01,
        3.6000e+01],
       ...,
       [1.7061e+04, 7.6630e+03, 7.2840e+03, ..., 3.2000e+01, 3.9200e+02,
        7.2000e+01],
       [4.7570e+03, 6.3450e+03, 6.0790e+03, ..., 2.5000e+01, 2.7500e+02,
        7.2000e+01],
       [3.4597e+04, 1.8877e+04, 1.8047e+04, ..., 1.3000e+01, 1.7000e+02,
        6.3000e+01]])

In [53]:
selected_features = pd.DataFrame(model.inverse_transform(X_new), 
                                 index=traindata_interaction.index,
                                 columns=traindata_interaction.columns)

# Dropped columns have values of all 0s, keep other columns 
selected_columns = selected_features.columns[selected_features.var() != 0]

In [54]:
selected_columns

Index(['LicenseNo', 'FacilityID', 'FacilityName', 'Type', 'Street', 'City',
       'State', 'LocationID', 'Reason', 'SectionViolations', 'RiskLevel',
       'Geo_Loc', 'day', 'month', 'year', 'ReasonRiskLevel',
       'ReasonSectionViolations', 'SectionViolationsRiskLevel'],
      dtype='object')

# Univariate featture selection 

In [56]:
from sklearn.feature_selection import SelectKBest, f_classif
selector = SelectKBest(f_classif, k=14)

X_new1 = selector.fit_transform(traindata_interaction, y_train)
X_new1


/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [12] are constant.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


array([[2.6121e+04, 7.6280e+03, 3.2600e+02, ..., 4.1000e+01, 4.9600e+02,
        7.2000e+01],
       [3.6379e+04, 9.2940e+03, 3.2600e+02, ..., 0.0000e+00, 2.6000e+01,
        7.6000e+01],
       [2.0219e+04, 4.6420e+03, 3.5100e+02, ..., 0.0000e+00, 1.3000e+01,
        3.6000e+01],
       ...,
       [1.7061e+04, 7.6630e+03, 3.2600e+02, ..., 3.2000e+01, 3.9200e+02,
        7.2000e+01],
       [4.7570e+03, 6.3450e+03, 3.2600e+02, ..., 2.5000e+01, 2.7500e+02,
        7.2000e+01],
       [3.4597e+04, 1.8877e+04, 3.2600e+02, ..., 1.3000e+01, 1.7000e+02,
        6.3000e+01]])

In [58]:
selected_features = pd.DataFrame(selector.inverse_transform(X_new1), 
                                 index=traindata_interaction.index, 
                                 columns=traindata_interaction.columns)
selected_features.head()

,LicenseNo,FacilityID,FacilityName,Type,Street,City,State,LocationID,Reason,SectionViolations,RiskLevel,Geo_Loc,hour,day,month,year,ReasonRiskLevel,ReasonSectionViolations,SectionViolationsRiskLevel
0,26121.0,7628.0,0.0,326.0,0.0,0.0,0.0,0.0,6.0,32.0,0.0,7682.0,0.0,4.0,7.0,2015.0,41.0,496.0,72.0
1,36379.0,9294.0,0.0,326.0,0.0,0.0,0.0,0.0,0.0,33.0,0.0,1912.0,0.0,19.0,9.0,2016.0,0.0,26.0,76.0
2,20219.0,4642.0,0.0,351.0,0.0,0.0,0.0,0.0,0.0,21.0,0.0,1352.0,0.0,22.0,1.0,2017.0,0.0,13.0,36.0
3,0.0,10843.0,0.0,64.0,0.0,0.0,0.0,0.0,2.0,3.0,0.0,5993.0,0.0,4.0,7.0,2015.0,28.0,333.0,63.0
4,3190.0,22923.0,0.0,326.0,0.0,0.0,0.0,0.0,1.0,32.0,0.0,15316.0,0.0,21.0,4.0,2015.0,25.0,275.0,72.0


# l2 penalty 

In [61]:
logistic2 = LogisticRegression(C=1, penalty="l2", random_state=7).fit(traindata_interaction, y_train)
model2 = SelectFromModel(logistic2, prefit=True)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [62]:
X_new = model2.transform(traindata_interaction)
X_new

array([[8.1846e+04, 3.2000e+01, 4.9600e+02, 7.2000e+01],
       [8.1865e+04, 3.3000e+01, 2.6000e+01, 7.6000e+01],
       [8.1897e+04, 2.1000e+01, 1.3000e+01, 3.6000e+01],
       ...,
       [8.1870e+04, 3.2000e+01, 3.9200e+02, 7.2000e+01],
       [8.1886e+04, 3.2000e+01, 2.7500e+02, 7.2000e+01],
       [8.1859e+04, 3.0000e+00, 1.7000e+02, 6.3000e+01]])

In [65]:
selected_features2 = pd.DataFrame(model2.inverse_transform(X_new), 
                                 index=traindata_interaction.index,
                                 columns=traindata_interaction.columns)

# Dropped columns have values of all 0s, keep other columns 
selected_columns2 = selected_features2.columns[selected_features2.var() != 0]

In [68]:
selected_columns2

Index(['LocationID', 'SectionViolations', 'ReasonSectionViolations',
       'SectionViolationsRiskLevel'],
      dtype='object')

In [76]:
train = traindata_interaction[selected_columns]

In [78]:
valid = testdata_interaction[selected_columns]

In [79]:
score_dataset(train, valid, y_train, y_valid)

              precision    recall  f1-score   support

     class 0       0.00      0.00      0.00         0
     class 1       0.23      0.26      0.24        23
     class 2       0.00      0.00      0.00         0
     class 3       0.00      0.00      0.00         4
     class 4       0.81      0.87      0.84        86
     class 5       0.44      0.48      0.46        23
     class 6       0.00      0.00      0.00        12

   micro avg       0.62      0.62      0.62       148
   macro avg       0.21      0.23      0.22       148
weighted avg       0.57      0.62      0.60       148



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.6216216216216216